In [4]:
from numpy import arange, hstack, log, array, full, cumsum, searchsorted, delete, unravel_index, logical_and, zeros, exp
from numpy.random import rand, choice
%load_ext line_profiler
def simulate_polymerase():
    end_position = 0
    no_domains = round(1.0 / (2.7e-3))
    has_npf_row = zeros(no_domains, dtype = bool)
    has_npf_row[arange(0, no_domains, 5)] = True
    has_monomer_row = zeros(no_domains, dtype = bool)
    monomer_count_row = zeros(no_domains, dtype = int)

    actin_conc = 5.0
    elongation_rate = 11 * actin_conc
    loading_rate = 5.0 * actin_conc * (42 / (42 + 8))**(1/3)
    unloading_rate = 3.0
    actin_diff_rate = 1.0 / (2.7e-3)**2

    current_time = 0.0
    total_time = 1.0

    transition_state_mat = zeros((4, no_domains))
    while current_time <= total_time and end_position <= no_domains:
        # Generate transition rate matrix.
        # Actin loading.
        transition_state_mat[0, logical_and(has_npf_row, ~has_monomer_row)] = loading_rate
        # Actin unloading.
        transition_state_mat[1, logical_and(has_npf_row, has_monomer_row)] = unloading_rate
        # Forward diffusion.
        transition_state_mat[2, :] = actin_diff_rate * monomer_count_row
        # Backward diffusion
        transition_state_mat[3, :] = actin_diff_rate * monomer_count_row
        total_rate = transition_state_mat.sum()
        #probability_row = transition_state_mat.flatten() / total_rate
        #index_transition = choice(probability_row.size, p = probability_row)
        transition_cumsum = cumsum(hstack((0, transition_state_mat.flatten())))
        index_transition = searchsorted(transition_cumsum > total_rate * rand(), True) - 1
        transition_row, transition_col = unravel_index(index_transition, transition_state_mat.shape)
        if transition_row == 0:
            has_monomer_row[transition_col] = True
        elif transition_row == 1:
            has_monomer_row[transition_col] = False
            monomer_count_row[transition_col] += 1
        elif transition_row == 2:
            monomer_count_row[transition_col] -= 1
            if transition_col < (no_domains - 1):
                monomer_count_row[transition_col + 1] += 1
        elif transition_row == 3:
            monomer_count_row[transition_col] -= 1
            if transition_col > 0:
                monomer_count_row[transition_col - 1] += 1
        time_interval = -log(rand()) / total_rate
        current_time += time_interval
        if monomer_count_row[end_position + 1] > 0:
            monomer_count_row[end_position + 1] -= 1
            end_position += 1
        if exp(-elongation_rate * time_interval) < rand():
            end_position += 1
    print(end_position / current_time)
    return end_position / current_time

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [5]:
%lprun -f simulate_polymerase simulate_polymerase()

182.99992926380608


Timer unit: 1e-06 s

Total time: 379.274 s
File: <ipython-input-4-6208e8f257ec>
Function: simulate_polymerase at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def simulate_polymerase():
     5         1          9.0      9.0      0.0      end_position = 0
     6         1         14.0     14.0      0.0      no_domains = round(1.0 / (2.7e-3))
     7         1         33.0     33.0      0.0      has_npf_row = zeros(no_domains, dtype = bool)
     8         1         54.0     54.0      0.0      has_npf_row[arange(0, no_domains, 5)] = True
     9         1         11.0     11.0      0.0      has_monomer_row = zeros(no_domains, dtype = bool)
    10         1         28.0     28.0      0.0      monomer_count_row = zeros(no_domains, dtype = int)
    11                                           
    12         1          4.0      4.0      0.0      actin_conc = 5.0
    13         1          6.0      6.0      0.0      elong